Total Cost Analysis for Manufacturing Facility Locations
# Overview
This notebook performs a comprehensive cost analysis for potential manufacturing facility locations, considering:
1. Labor costs
2. Shipping rates (FTL)
3. Rent and utilities costs
4. Distance to demand centers

# Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import describe

# Load and Prepare Data

In [ ]:
# Load all relevant datasets
rates_df = pd.read_csv('Data/rates.csv')
stores_df = pd.read_csv('Data/stores.csv')
demand_df = pd.read_csv('Data/demand.csv')
products_df = pd.read_csv('Data/products.csv')
labor_df = pd.read_csv('Data/labor_cost.csv')
facilities_df = pd.read_csv('Data/facilities.csv')

# Calculate annual demand in pallets
demand_df = demand_df.merge(products_df[['SKU', 'SKUs/pallet']], on='SKU', how='left')
demand_df['Demand_Pallets'] = demand_df['Demand'] / demand_df['SKUs/pallet']
annual_demand = demand_df.groupby('StoreID')['Demand_Pallets'].sum().reset_index()
store_demand = stores_df.merge(annual_demand, on='StoreID', how='left')

# Display summary of each dataset
print("Labor Costs Summary:")
display(labor_df.describe())
print("\nFacilities Cost Summary:")
display(facilities_df.describe())

# Analyze Labor Costs

In [ ]:
# Visualize labor costs by region
plt.figure(figsize=(12, 6))
labor_costs = labor_df.sort_values('Local Labor Cost (USD/hour)', ascending=True)
plt.bar(labor_costs['Location'], labor_costs['Local Labor Cost (USD/hour)'])
plt.xticks(rotation=45, ha='right')
plt.title('Labor Costs by Location')
plt.xlabel('Location')
plt.ylabel('Labor Cost (USD/hour)')
plt.tight_layout()
plt.show()

# Analyze Facility Costs

In [ ]:
# Calculate total facility cost per square foot
facilities_df['Total_Cost_SqFt'] = facilities_df['RentCost_SqFt'] + facilities_df['UtilitiesOpsCost_SqFt']

# Create visualization of facility costs
plt.figure(figsize=(12, 6))
facilities_sorted = facilities_df.sort_values('Total_Cost_SqFt', ascending=True)
plt.bar(facilities_sorted['Location'], facilities_sorted['Total_Cost_SqFt'])
plt.xticks(rotation=45, ha='right')
plt.title('Total Facility Costs by Location')
plt.xlabel('Location')
plt.ylabel('Total Cost per Square Foot (USD)')
plt.tight_layout()
plt.show()

# Shipping Cost Analysis

In [ ]:
def calculate_pallet_mile_rate(row):
    """Calculate shipping cost per pallet per mile"""
    # Assuming standard 53' trailer can hold 26 pallets
    pallets_per_truck = 26
    
    # Calculate distance between origin and destination
    origin_coords = get_coords(row['Origin'])
    dest_coords = get_coords(row['Destination'])
    distance = geodesic(origin_coords, dest_coords).miles
    
    # Calculate rate per pallet-mile
    pallet_mile_rate = row['Cost_FTL'] / (pallets_per_truck * distance)
    return pallet_mile_rate, distance

def get_coords(city_state):
    """Get coordinates for a city from our stores dataset"""
    city = city_state.split(',')[0].strip()
    state = city_state.split(',')[1].strip()
    
    # Find matching city in stores dataset
    matching_stores = stores_df[
        (stores_df['City'] == city) & 
        (stores_df['State'] == state)
    ]
    
    if len(matching_stores) > 0:
        return (matching_stores.iloc[0]['Latitude'], matching_stores.iloc[0]['Longitude'])
    
    # If not in stores_df, try facilities_df
    matching_facilities = facilities_df[facilities_df['Location'] == f"{city}, {state}"]
    if len(matching_facilities) > 0:
        # You would need lat/long in facilities_df for this to work
        return (matching_facilities.iloc[0]['Latitude'], matching_facilities.iloc[0]['Longitude'])
    
    return None

# Calculate rates and distances
rates_df[['Rate_Per_Pallet_Mile', 'Distance']] = pd.DataFrame(
    rates_df.apply(calculate_pallet_mile_rate, axis=1).tolist(),
    index=rates_df.index
)

# Analyze shipping rates
plt.figure(figsize=(10, 6))
plt.scatter(rates_df['Distance'], rates_df['Rate_Per_Pallet_Mile'])
plt.title('Shipping Rates vs Distance')
plt.xlabel('Distance (miles)')
plt.ylabel('Rate ($/pallet-mile)')
plt.show()

# Target Region Analysis
Based on our previous clustering analysis, we'll analyze costs around:
1. Central California (around 37.35°N, 118.62°W)
2. West Virginia (around 38.88°N, 80.70°W)
3. Northern Texas (around 33.68°N, 97.72°W)

In [ ]:
target_regions = {
    'Central California': (37.3506, -118.6246),
    'West Virginia': (38.8837, -80.6994),
    'Northern Texas': (33.6814, -97.7189)
}

def analyze_region_costs(region_name, region_coords):
    """Analyze all costs for a specific region"""
    
    # 1. Find nearest labor market
    labor_costs = []
    for _, row in labor_df.iterrows():
        coords = get_coords(row['Location'])
        if coords:
            dist = geodesic(region_coords, coords).miles
            if dist < 300:  # Consider labor markets within 300 miles
                labor_costs.append(row['Local Labor Cost (USD/hour)'])
    
    avg_labor_cost = np.mean(labor_costs) if labor_costs else None
    
    # 2. Find nearby facility costs
    facility_costs = []
    for _, row in facilities_df.iterrows():
        coords = get_coords(row['Location'])
        if coords:
            dist = geodesic(region_coords, coords).miles
            if dist < 300:
                facility_costs.append(row['Total_Cost_SqFt'])
    
    avg_facility_cost = np.mean(facility_costs) if facility_costs else None
    
    # 3. Calculate average shipping rates
    shipping_rates = []
    for _, row in rates_df.iterrows():
        origin_coords = get_coords(row['Origin'])
        dest_coords = get_coords(row['Destination'])
        
        if origin_coords and dest_coords:
            dist_to_origin = geodesic(region_coords, origin_coords).miles
            dist_to_dest = geodesic(region_coords, dest_coords).miles
            
            if dist_to_origin < 300 or dist_to_dest < 300:
                shipping_rates.append(row['Rate_Per_Pallet_Mile'])
    
    avg_shipping_rate = np.mean(shipping_rates) if shipping_rates else None
    
    return {
        'labor_cost': avg_labor_cost,
        'facility_cost': avg_facility_cost,
        'shipping_rate': avg_shipping_rate
    }

# Analyze each region
results = {}
for region, coords in target_regions.items():
    results[region] = analyze_region_costs(region, coords)

# Create comparison visualizations
regions = list(results.keys())
metrics = ['labor_cost', 'facility_cost', 'shipping_rate']
titles = ['Labor Cost ($/hour)', 'Facility Cost ($/sqft)', 'Shipping Rate ($/pallet-mile)']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
for i, (metric, title) in enumerate(zip(metrics, titles)):
    values = [results[region][metric] for region in regions]
    axes[i].bar(regions, values)
    axes[i].set_title(title)
    axes[i].tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

# Total Cost Analysis

In [ ]:
# Define assumptions
working_hours_per_year = 2080  # 40 hours/week * 52 weeks
required_sqft = 50000  # Example facility size
workers_needed = 50  # Example number of workers
annual_total_pallets = store_demand['Demand_Pallets'].sum()

# Calculate annual costs for each region
annual_costs = {}
for region in results:
    labor_cost = results[region]['labor_cost'] * workers_needed * working_hours_per_year
    facility_cost = results[region]['facility_cost'] * required_sqft
    
    # Estimate average shipping distance and cost
    avg_distance = 500  # Example average distance to stores
    shipping_cost = results[region]['shipping_rate'] * annual_total_pallets * avg_distance
    
    annual_costs[region] = {
        'Labor Cost': labor_cost,
        'Facility Cost': facility_cost,
        'Estimated Shipping Cost': shipping_cost,
        'Total Cost': labor_cost + facility_cost + shipping_cost
    }

# Create cost comparison visualization
cost_df = pd.DataFrame(annual_costs).T
cost_df_melted = cost_df.reset_index().melt(id_vars=['index'], var_name='Cost Type', value_name='Amount')

plt.figure(figsize=(12, 6))
sns.barplot(data=cost_df_melted, x='index', y='Amount', hue='Cost Type')
plt.title('Annual Cost Comparison by Region')
plt.xlabel('Region')
plt.ylabel('Annual Cost (USD)')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Print detailed cost breakdown
print("\nDetailed Annual Cost Breakdown (in millions USD):")
for region, costs in annual_costs.items():
    print(f"\n{region}:")
    for cost_type, amount in costs.items():
        print(f"{cost_type}: ${amount/1_000_000:.2f}M")

# Conclusions and Recommendations

Based on our analysis:

1. Labor Costs:
   - Highest in [region with highest labor cost]
   - Lowest in [region with lowest labor cost]

2. Facility Costs:
   - Highest in [region with highest facility cost]
   - Lowest in [region with lowest facility cost]

3. Shipping Costs:
   - Most efficient from [region with lowest shipping rate]
   - Highest from [region with highest shipping rate]

4. Total Cost Comparison:
   - Most cost-effective region: [region with lowest total cost]
   - Most expensive region: [region with highest total cost]

Recommendations:
1. Primary consideration: [main recommendation]
2. Secondary options: [alternative options]
3. Trade-offs to consider: [list key trade-offs]